In [1]:
import logging
#Logging is a means of tracking events that happen when some software runs
# Create and configure logger
logging.basicConfig(filename="../logs/newfile.log",
                    format='%(asctime)s %(message)s',
                    filemode='w')
 
# Creating an object
logger = logging.getLogger()
 
# Setting the threshold of logger to DEBUG
logger.setLevel(logging.DEBUG)
 
# Test messages
logger.debug("Harmless debug Message")
logger.info("Just an information")
logger.warning("Its a Warning")
logger.error("Did you try to divide by zero")
logger.critical("Internet is down")

In [2]:
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
import logging
import warnings
warnings.filterwarnings("ignore")

sns.set_style("darkgrid")
%matplotlib inline
import sys
sys.path.insert(0,'../scripts/')

C:\Users\kachase\Anaconda3\lib\site-packages\pandas\compat\_optional.py:138: UserWarning: Pandas requires version '2.7.0' or newer of 'numexpr' (version '2.6.9' currently installed).
  warnings.warn(msg, UserWarning)


In [4]:
from data_loader import load_df_from_csv
from data_struct import DataInfo
from cleaner import DataCleaner
from prophet import prophet as Prophet
import pandas as pd
import warnings

ModuleNotFoundError: No module named 'prophet'

In [ ]:
df = pd.read_csv("../data/train_processed.csv")

test = pd.read_csv("../data/test_processed.csv")

In [ ]:
sales = pd.DataFrame(df['Sales'])
sales['Date'] = df['Date']

sales.head()

In [ ]:
sales.dtypes

In [ ]:
sales.rename(columns = {'Date':'ds', 'Sales':'y'}, inplace = True)
data = sales.groupby("ds").agg({"y": "mean"})
data

In [ ]:
sales['ds'] = pd.to_datetime(filtered['ds'])
sales.dtypes
data = sales.groupby("ds").agg({"y": "mean"})
data
sample = sales[0:7000]

### Train Model

In [ ]:
m = Prophet.Prophet(interval_width=0.95, daily_seasonality= True)
model = m.fit(sample)

### Forcasting

In [ ]:
future = m.make_future_dataframe(periods=100, freq="D")
forecast = m.predict(future)
forecast.head()

In [ ]:
forecast[['ds', 'yhat']]

In [ ]:
plot1 = m.plot(forecast)

In [ ]:
plot2 = m.plot_components(forecast)

### modelling for holidays as well

In [ ]:
# create holidays dataframe
state_dates = df[(df.StateHoliday == 'a') | (df.StateHoliday == 'b') & (df.StateHoliday == 'c')].loc[:, 'Date'].values
school_dates = df[df.SchoolHoliday == 1].loc[:, 'Date'].values

state = pd.DataFrame({'holiday': 'state_holiday',
                      'ds': pd.to_datetime(state_dates)})
school = pd.DataFrame({'holiday': 'school_holiday',
                      'ds': pd.to_datetime(school_dates)})

holidays = pd.concat((state, school))      
holidays.head()

In [ ]:
# set the uncertainty interval to 95% (the Prophet default is 80%)
my_model = Prophet(interval_width = 0.95, 
                   holidays = holidays)
my_model.fit()

# dataframe that extends into future 6 weeks 
future_dates = my_model.make_future_dataframe(periods = 6*7)

print("First week to forecast.")
future_dates.tail(7)

In [ ]:
# predictions
forecast = my_model.predict(future_dates)

# preditions for last week
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']].tail(7)

In [ ]:
fc = forecast[['ds', 'yhat']].rename(columns = {'Date': 'ds', 'Forecast': 'yhat'})

In [ ]:
# visualizing predicions
my_model.plot(forecast);

In [ ]:
my_model.plot_components(forecast);